# 1. Import of input data

In [1]:
import nbimporter
import sys
import os
from pathlib import Path

In [2]:
# Import libraries
try:
    current_dir = RUN_DIR
    print(f"📂 CoolingCompressionChiller.ipynb executed by RUN.ipynb — using directory: {current_dir}")
except NameError:
    current_dir = Path().resolve()
    print(f"📁 CoolingCompressionChiller.ipynb executed standalone — using directory: {current_dir}")

Libraries_path = (current_dir / "Libraries.ipynb").resolve()

# If not found, try in the parent levels
if not Libraries_path.exists():
    Libraries_path = (current_dir.parent / "Libraries.ipynb").resolve()

# Run the file
if Libraries_path.exists():
    print(f"📘 Importing libraries from: {Libraries_path}")
    get_ipython().run_line_magic('run', f'"{Libraries_path}"')
else:
    print("⚠️ Libraries.ipynb not found! Some functions might not be available.")

📁 CoolingCompressionChiller.ipynb executed standalone — using directory: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Energy_systems
📘 Importing libraries from: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw\Libraries.ipynb
📁 Libraries.ipynb — directory impostata a: C:\Users\RELab\OneDrive - Politecnico di Milano\grupporelab\02_Progetti\AREA LCA\Kevin\PhD\aa20232024\03_Sviluppo\Nuovo modello\FINALE\bw


In [3]:
# Database parameter import
energysystem = pd.read_excel(energysystem_path, sheet_name = 'Cooling - Compression chiller')

# 2. Input parameters

In [4]:
# DASHBORD - VALUE INPUTS
Power = energysystem['Cooling capacity [kW]']
EnergyCooling = energysystem['Energy produced [kWh /year]']
SCOP = energysystem['COP']
ElectricityGrid = energysystem['Energy input from national grid [kWh/year]']
ElectricityPV = energysystem['Energy input from Photovoltaic system [kWh/year]']
ElectricityRICE = energysystem['Energy input from CHP-Internal Combustion Engine [kWh/year]']
ElectricityCombinedCycle = energysystem['Energy input from CHP-Combined cycle [kWh/year]']
ElectricityGasTurbine = energysystem['Energy input from CHP-Gas Turbine [kWh/year]']
ElectricityWtE = energysystem['Energy input from CHP-Waste-to-Energy [kWh/year]']
ElectricityORC = energysystem['Energy input from CHP-Organic Rankine Cycle [kWh/year]']

In [5]:
# DASHBOARD - INPUT PARAMETERS
massRef = 3500 #kg
PowerRef = 300 # kW (Source: Ecoinvent database)
mass = massRef * (Power / PowerRef)
Lifespan = 15 #years

# 3. Life cycle inventories

## 3.1 Module A1-3

In [6]:
CompressionChiller_aluminium = 0.0427 * mass
CompressionChiller_castiron = 0.1921 * mass
CompressionChiller_copper = 0.0138 * mass
CompressionChiller_lowalloyedsteel = 0.7515 * mass
CompressionChiller_sheetrollingsteel = 0.7515 * mass
CompressionChiller_sheetrollingaluminium = 0.0427 * mass

## 3.2 Module A4

In [7]:
CompressionChiller_Truk_16_32 = ((500 + 200)*(mass)/1000)
CompressionChiller_Light_Truk = (50*(mass)/1000)

## 3.3 Module B2

In [8]:
CompressionChiller_Light_Truk_B2 = 50 * (Lifespan/2) * 0.01 * (mass/1000)

## 3.4 Module B4

In [9]:
Lifespan = 25 #year Hampoo et al. (2021)
ReqSL = RSP 
CompressionChillerNumberReplacements = (np.ceil(np.divide(ReqSL, Lifespan) - 1))
SFCompressionChiller = (RSP / ReqSL)  # Scaling factor

## 3.5 Module B6

In [10]:
#Parameters for energy input repartition
TotalElectricityInput = (ElectricityGrid + ElectricityPV + ElectricityRICE + ElectricityCombinedCycle +
    ElectricityGasTurbine + ElectricityWtE + ElectricityORC)

CompressionChiller_EleGrid = ElectricityGrid / TotalElectricityInput
CompressionChiller_ElePV = ElectricityPV / TotalElectricityInput
CompressionChiller_EleRICE = ElectricityRICE / TotalElectricityInput
CompressionChiller_EleCombinedCycle = ElectricityCombinedCycle / TotalElectricityInput
CompressionChiller_EleGasTurbine = ElectricityGasTurbine / TotalElectricityInput
CompressionChiller_EleWtE = ElectricityWtE / TotalElectricityInput
CompressionChiller_EleORC = ElectricityORC / TotalElectricityInput

In [11]:
CompressionChiller_usephaseelectricity = (1 / SCOP) * EnergyCooling * RSP * CompressionChiller_EleGrid
CompressionChiller_usephaseelectricityPV = (1 / SCOP) * EnergyCooling * RSP * CompressionChiller_ElePV

## 3.6 Module C2

In [12]:
CompressionChiller_Transport_to_treatment_plant = 50 * (mass/1000)

## 3.7 Module C3

In [13]:
CompressionChiller_EOL_aluminium = CompressionChiller_aluminium
CompressionChiller_EOL_copper = CompressionChiller_copper
CompressionChiller_EOL_castiron = CompressionChiller_castiron
CompressionChiller_EOL_lowalloyedsteel = CompressionChiller_lowalloyedsteel

## 3.8 Module C4

In [14]:
CompressionChiller_EOL_landfill_aluminium = CompressionChiller_EOL_aluminium * LandfillMetals
CompressionChiller_EOL_landfill_copper = CompressionChiller_EOL_copper * LandfillMetals
CompressionChiller_EOL_landfill_lowalloyedsteel = CompressionChiller_EOL_lowalloyedsteel * LandfillMetals
CompressionChiller_EOL_landfill_castiron = CompressionChiller_EOL_castiron * LandfillMetals

## 3.9 Module D1 (related to the export of secondary materials)

In [15]:
CompressionChiller_EOL_recycling_castiron = (RecyclingMetals2 * RecyclingEfficiencyMetals * CompressionChiller_EOL_castiron) - (RecycledContentMetals * CompressionChiller_EOL_castiron)
CompressionChiller_EOL_primary_castiron = ((RecycledContentMetals * CompressionChiller_EOL_castiron) - (RecyclingMetals2 * RecyclingEfficiencyMetals * CompressionChiller_EOL_castiron)) * SRmetals

CompressionChiller_EOL_recycling_steel = (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_lowalloyedsteel) - (RecycledContentMetals * CompressionChiller_EOL_lowalloyedsteel)
CompressionChiller_EOL_primary_steel = ((RecycledContentMetals * CompressionChiller_EOL_lowalloyedsteel) - (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_lowalloyedsteel)) * SRmetals

CompressionChiller_EOL_recycling_aluminium = (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_aluminium) - (RecycledContentMetals * CompressionChiller_EOL_aluminium)
CompressionChiller_EOL_primary_aluminium = ((RecycledContentMetals * CompressionChiller_EOL_aluminium) - (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_aluminium)) * SRmetals

CompressionChiller_EOL_recycling_copper = (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_copper) - (RecycledContentMetals * CompressionChiller_EOL_copper)
CompressionChiller_EOL_primary_copper = ((RecycledContentMetals * CompressionChiller_EOL_copper) - (RecyclingMetals * RecyclingEfficiencyMetals * CompressionChiller_EOL_copper)) * SRmetals


# 4. Data for export 

In [16]:
ThermalEnergyAbsorbed_CompressionChiller = EnergyCooling * RSP